In [11]:
!pip install yfinance sqlalchemy statsmodels lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 46.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)
sp500_table = tables[0]

print("Successfully extracted table from Wikipedia")
print(f"Table shape: {sp500_table.shape}")
print(f"Columns: {list(sp500_table.columns)}\n")

Successfully extracted table from Wikipedia
Table shape: (503, 8)
Columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']



In [14]:
# Extract tickers and clean them
tickers = sp500_table['Symbol'].str.replace('.', '-').tolist()
print(f"Total number of tickers extracted: {len(tickers)}")
print(f"First 10 tickers: {tickers[:10]}")
print(f"Last 10 tickers: {tickers[-10:]}\n")

Total number of tickers extracted: 503
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']
Last 10 tickers: ['WMB', 'WTW', 'WDAY', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZTS']



In [15]:
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

print(f"Date range: {start_date.date()} to {end_date.date()}")

Date range: 2022-10-19 to 2025-10-18


In [16]:
sp500_index = yf.download('^GSPC', start=start_date, end=end_date, progress=False)
print(f"✓ S&P 500 Index downloaded: {len(sp500_index)} days of data\n")

✓ S&P 500 Index downloaded: 752 days of data



In [17]:
stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=True)

print(f"Downloaded: {stock_data.shape}")

[*********************100%***********************]  503 of 503 completed


Downloaded: (752, 2515)


In [19]:
# Step 4: Extract Adjusted Close Prices and Calculate Returns
# Reference: Course materials on return calculation
print("="*60)
print("STEP 4: Extracting Prices and Calculating Returns")
print("="*60)

# Extract Close prices from stocks
print("Extracting 'Close' prices from MultiIndex columns...")
adj_close_prices = stock_data.xs('Close', level=1, axis=1)

print(f"✓ Close prices extracted: {adj_close_prices.shape}")
print(f"  Tickers: {adj_close_prices.shape[1]}")
print(f"  Days: {adj_close_prices.shape[0]}")
print(f"  First 5 tickers: {adj_close_prices.columns[:5].tolist()}")
print(f"  Date range: {adj_close_prices.index[0].date()} to {adj_close_prices.index[-1].date()}")

# Calculate returns
print("\nCalculating returns...")
returns = adj_close_prices.pct_change().dropna()

# For S&P 500, use 'Close' column
sp500_returns = sp500_index['Close'].pct_change().dropna()

print(f"✓ Stock returns calculated: {returns.shape}")
print(f"✓ S&P 500 returns calculated: {sp500_returns.shape}")
print(f"  Date range: {returns.index[0].date()} to {returns.index[-1].date()}")

# Show sample of returns
print(f"\nSample stock returns (first 3 tickers, first 3 days):")
print(returns.iloc[:3, :3])

print(f"\nSample S&P 500 returns (first 5 days):")
print(sp500_returns.head())

print("="*60 + "\n")

STEP 4: Extracting Prices and Calculating Returns
Extracting 'Close' prices from MultiIndex columns...
✓ Close prices extracted: (752, 503)
  Tickers: 503
  Days: 752
  First 5 tickers: ['ALLE', 'CCL', 'PAYX', 'NDAQ', 'APD']
  Date range: 2022-10-19 to 2025-10-17

Calculating returns...
✓ Stock returns calculated: (391, 503)
✓ S&P 500 returns calculated: (751, 1)
  Date range: 2024-03-28 to 2025-10-17

Sample stock returns (first 3 tickers, first 3 days):
Ticker          ALLE       CCL      PAYX
Date                                    
2024-03-28  0.003501 -0.049447  0.010450
2024-04-01 -0.017742 -0.004896 -0.009609
2024-04-02 -0.005593 -0.047971  0.003782

Sample S&P 500 returns (first 5 days):
Ticker         ^GSPC
Date                
2022-10-20 -0.007951
2022-10-21  0.023725
2022-10-24  0.011882
2022-10-25  0.016267
2022-10-26 -0.007388



In [20]:
db_name = 'sp500_data.db'
engine = create_engine(f'sqlite:///{db_name}')

# Store data in database
adj_close_prices.to_sql('adj_close_prices', engine, if_exists='replace', index=True)
print(f"Stored adj_close_prices: {adj_close_prices.shape}")

returns.to_sql('returns', engine, if_exists='replace', index=True)
print(f"Stored returns: {returns.shape}")

sp500_index.to_sql('sp500_index', engine, if_exists='replace', index=True)
print(f"Stored sp500_index: {sp500_index.shape}")

sp500_returns.to_sql('sp500_returns', engine, if_exists='replace', index=True)

Stored adj_close_prices: (752, 503)
Stored returns: (391, 503)
Stored sp500_index: (752, 5)


751

In [24]:
# Standardize returns
scaler = StandardScaler()
returns_standardized = scaler.fit_transform(returns)
print(f"Data standardized")

# Apply PCA with 5 components
pca = PCA(n_components=5)
principal_components = pca.fit_transform(returns_standardized)
print(f"PCA completed: {principal_components.shape}\n")

# Variance explained
variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

print("Variance Explained by Each Component:")
print("-" * 50)
for i in range(5):
    print(f"  PC{i+1}: {variance_explained[i]*100:.2f}% (Cumulative: {cumulative_variance[i]*100:.2f}%)")
print("-" * 50)
print(f"Total: {cumulative_variance[-1]*100:.2f}%\n")

# Create DataFrame with PC returns
pc_returns = pd.DataFrame(
    principal_components,
    index=returns.index,
    columns=[f'PC{i+1}' for i in range(5)]
)
print(f"PC returns DataFrame created: {pc_returns.shape}")
print("="*60 + "\n")

Data standardized
PCA completed: (391, 5)

Variance Explained by Each Component:
--------------------------------------------------
  PC1: 28.71% (Cumulative: 28.71%)
  PC2: 8.76% (Cumulative: 37.47%)
  PC3: 3.61% (Cumulative: 41.07%)
  PC4: 2.52% (Cumulative: 43.60%)
  PC5: 1.96% (Cumulative: 45.55%)
--------------------------------------------------
Total: 45.55%

PC returns DataFrame created: (391, 5)



In [25]:
common_dates = pc_returns.index.intersection(sp500_returns.index)
pc_returns_aligned = pc_returns.loc[common_dates]
sp500_returns_aligned = sp500_returns.loc[common_dates]

print(f"Aligned data: {len(common_dates)} dates\n")

# Run regressions for each PC
regression_results = {}

for i in range(1, 6):
    pc_name = f'PC{i}'
    print(f"{pc_name} Regression:")
    print("-" * 50)

    # Prepare regression data
    X = sm.add_constant(sp500_returns_aligned.values)  # Add intercept
    y = pc_returns_aligned[pc_name].values

    # Fit OLS model
    model = sm.OLS(y, X).fit()

    # Store results
    regression_results[pc_name] = {
        'alpha': model.params[0],
        'beta': model.params[1],
        'alpha_pvalue': model.pvalues[0],
        'beta_pvalue': model.pvalues[1],
        'r_squared': model.rsquared
    }

    # Display results
    print(f"Alpha: {model.params[0]:.6f} (p-value: {model.pvalues[0]:.4f})")
    print(f"Beta:  {model.params[1]:.6f} (p-value: {model.pvalues[1]:.4f})")
    print(f"R^2:    {model.rsquared:.4f}\n")

print("="*60)
print("Summary Table:")
print("="*60)
summary_df = pd.DataFrame({
    'PC': [f'PC{i}' for i in range(1, 6)],
    'Alpha': [regression_results[f'PC{i}']['alpha'] for i in range(1, 6)],
    'Beta': [regression_results[f'PC{i}']['beta'] for i in range(1, 6)],
    'Alpha p-val': [regression_results[f'PC{i}']['alpha_pvalue'] for i in range(1, 6)],
    'Beta p-val': [regression_results[f'PC{i}']['beta_pvalue'] for i in range(1, 6)],
    'R^2': [regression_results[f'PC{i}']['r_squared'] for i in range(1, 6)]
})
print(summary_df.to_string(index=False))

Aligned data: 391 dates

PC1 Regression:
--------------------------------------------------
Alpha: -0.663319 (p-value: 0.0197)
Beta:  992.895000 (p-value: 0.0000)
R^2:    0.7848

PC2 Regression:
--------------------------------------------------
Alpha: 0.122552 (p-value: 0.7037)
Beta:  -183.442890 (p-value: 0.0000)
R^2:    0.0878

PC3 Regression:
--------------------------------------------------
Alpha: -0.035965 (p-value: 0.8669)
Beta:  53.834599 (p-value: 0.0073)
R^2:    0.0184

PC4 Regression:
--------------------------------------------------
Alpha: 0.039081 (p-value: 0.8265)
Beta:  -58.498992 (p-value: 0.0005)
R^2:    0.0310

PC5 Regression:
--------------------------------------------------
Alpha: 0.001796 (p-value: 0.9910)
Beta:  -2.688952 (p-value: 0.8563)
R^2:    0.0001

Summary Table:
 PC     Alpha        Beta  Alpha p-val    Beta p-val      R^2
PC1 -0.663319  992.895000     0.019655 7.560547e-132 0.784847
PC2  0.122552 -183.442890     0.703726  2.287727e-09 0.087823
PC3 -0.0